# CL Housing Analysis

# Part 1: Data scraping and preparation

# _*** From project 1 of CMSC641_
### Let's do some scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
r = requests.get('https://washingtondc.craigslist.org/search/mld/apa?search_distance=10&postal=20740&availabilityMode=0&sale_date=all+dates')
r.text;
soup = BeautifulSoup(r.content, 'lxml');
soup

<!DOCTYPE html>
<html class="no-js"><head>
<title>washington, DC apts/housing for rent  - craigslist</title>
<meta content="washington, DC apts/housing for rent  - craigslist" name="description"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<link href="https://washingtondc.craigslist.org/search/mld/apa" rel="canonical"/>
<link href="https://washingtondc.craigslist.org/search/mld/apa?availabilityMode=0&amp;format=rss&amp;postal=20740&amp;search_distance=10" rel="alternate" title="RSS feed for craigslist | washington, DC apts/housing for rent  - craigslist" type="application/rss+xml"/>
<link href="https://washingtondc.craigslist.org/search/mld/apa?s=120&amp;availabilityMode=0&amp;postal=20740&amp;search_distance=10" rel="next"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="//www.craigslist.org/styles/cl.css?v=2c141c904c21cf0b362b8a9ce62cf33c" media="all" rel="stylesheet" type="text/css"/>
<link href="//www.craigslist.org/styles/search.css?v=

In [3]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js">
 <head>
  <title>
   washington, DC apts/housing for rent  - craigslist
  </title>
  <meta content="washington, DC apts/housing for rent  - craigslist" name="description"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <link href="https://washingtondc.craigslist.org/search/mld/apa" rel="canonical"/>
  <link href="https://washingtondc.craigslist.org/search/mld/apa?availabilityMode=0&amp;format=rss&amp;postal=20740&amp;search_distance=10" rel="alternate" title="RSS feed for craigslist | washington, DC apts/housing for rent  - craigslist" type="application/rss+xml"/>
  <link href="https://washingtondc.craigslist.org/search/mld/apa?s=120&amp;availabilityMode=0&amp;postal=20740&amp;search_distance=10" rel="next"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <link href="//www.craigslist.org/styles/cl.css?v=2c141c904c21cf0b362b8a9ce62cf33c" media="all" rel="stylesheet" type="text/css"/>
  <link href="//www.craigsli

Looking at the source code after searching for all apts/housing for rent within 10 miles of CP, we see that every entry on the map starts with <li class="result-row" data-pid="675xxxxxxx".

In [63]:
# LETS GET SOME ATTRIBUTES
# Data retrieved from GET request above only yields 120 rows?!

'''<a class="result-title hdrlnk" data-id="6766765340" href="https://washingtondc.craigslist.org/doc/apa/d/beautiful-jr-one-bedroom/6766765340.html">
         A Beautiful Jr. One Bedroom Condo in Sought After Broighton Building
        </a>
        <time class="result-date" datetime="2018-12-06 22:58" title="Thu 06 Dec 10:58:14 PM">Dec  6</time>
        '''
sum_Unlabeled = 0 
r_matrix = []
for row in soup.find_all(class_="result-row"):
    #Try is to avoid Nonetype object error (can't call .text)
    try:
        pID = row.get('data-pid')

        rtitle = row.find(class_="result-title hdrlnk") #Grabs post title + href
        rtitle_txt=rtitle.text
        href = rtitle.get('href')
        date = row.find(class_="result-date").text.get('datetime')
        print(date)
        r = row.find(class_='housing').text.splitlines() # for each row, grab BR + SQFT
        
        #Removes whitespace/empty lines
        brSqft = [i.replace(" ",'').replace('-','') for i in r if not (i.isspace()) and i != '']
        #print(rcl)
    except: 
        sum_Unlabeled+=1 #Catch value error, some don't have text
    r_matrix.append([pID, rtitle_txt, href] + brSqft)
#print(r_matrix[:10])
df = pd.DataFrame(r_matrix)
print(df)


              0                                                  1  \
0    6759804303   Hardwood Flooring, Spacious Floorplans, Dog Park   
1    6759009217  Corner Three Bedroom Available, Full Size W/D,...   
2    6744703743  Over-Sized 2 Bedroom 1.5 Bath W/D Inside Amazi...   
3    6747400318       "Bicycle storage and maintenance facilities"   
4    6760088996  24 Hour Front Desk, Walk to PG Plaza Metro Rai...   
5    6745487927                 Pool, Cable Ready, Laundry Service   
6    6758270265  Residents' clubhouse, Most utilities included,...   
7    6743084722    Playground, Sundeck, Resort-Style Swimming Pool   
8    6766799774  Newseum Residences | Where luxury city living ...   
9    6763550135  Off-street parking, Onsite maintenance, Short-...   
10   6752882180  Apply for $9.99 online!!! STAINLESS STEEL APPL...   
11   6747218748                 1 Bed with great sunlight exposure   
12   6759662791  GAS COOKING, WALL-TO-WALL CARPETING, IMPRESSIV...   
13   6755446990     

# PART 2: Storing in SQL (super basic)

In [1]:
#Super basic example of how to write to a SQLite DB
from sqlalchemy import create_engine
engine = create_engine('sqlite:///foo.db') #Already created
connection = engine.connect()

#print(engine.execute("SELECT * FROM Listings").fetchall())

df.to_sql('Listings', con=engine,if_exists='replace')
connection.close()
#Fetch all Listings results [ID, PID, Title, href, BR, Sqft]
#engine.execute("SELECT * FROM Listings").fetchall()

NameError: name 'df' is not defined

In [43]:
#Example ROW HTML CL
<li class="result-row" data-pid="6766765340">
       <a class="result-image gallery" data-ids="1:00T0T_l3wW2i19dU4,1:00808_27z8dUOmSMc,1:00R0R_kZgzYnaiyS1,1:00F0F_6vlKEH1eVly,1:00505_iYmECZZDUSu,1:00B0B_d8JLqKVfaT9,1:00E0E_4gBiNtn6kcx,1:00Y0Y_4LZ3Igh2TzK,1:00g0g_eEcoaDN6LMl,1:00404_xaChvWCrwb,1:00p0p_jLnMTuiLRnR" href="https://washingtondc.craigslist.org/doc/apa/d/beautiful-jr-one-bedroom/6766765340.html">
        <span class="result-price">
         $1900
        </span>
       </a>
       <p class="result-info">
        <span class="icon icon-star" role="button">
         <span class="screen-reader-text">
          favorite this post
         </span>
        </span>
        <time class="result-date" datetime="2018-12-06 23:50" title="Thu 06 Dec 11:50:20 PM">
         Dec  6
        </time>
        <a class="result-title hdrlnk" data-id="6766765340" href="https://washingtondc.craigslist.org/doc/apa/d/beautiful-jr-one-bedroom/6766765340.html">
         A Beautiful Jr. One Bedroom Condo in Sought After Broighton Building
        </a>
        <span class="result-meta">
         <span class="result-price">
          $1900
         </span>
         <span class="housing">
          1br -
                    600ft
          <sup>
           2
          </sup>
          -
         </span>
         <span class="result-hood">
          (Kalorama/ Dupont Circle)
         </span>
         <span class="result-tags">
          pic
     <span class="maptag" data-pid="6766765340">
           map
          </span>
         </span>
         <span class="banish icon icon-trash" role="button">
          <span class="screen-reader-text">
           hide this posting
          </span>
         </span>
         <span aria-hidden="true" class="unbanish icon icon-trash red" role="button">
         </span>
         <a class="restore-link" href="#">
          <span class="restore-narrow-text">
           restore
          </span>
          <span class="restore-wide-text">
           restore this posting
          </span>
         </a>
        </span>
       </p>
      </li>

SyntaxError: invalid syntax (<ipython-input-43-4148719efacc>, line 2)

## Step 2: Tidying
## Here we tidy the data as required. We combine the date and time columns for start, end and max, into single datetime columns through the function makeDateTime was used to parse these, and then the datetime package was used to combine these into datetime objects. We also drop the unneeded columns and set missing values to NaN.

## Step 3: Scrap NASA data
## Here we scrape the data from the nasa site, similarly as before, neglecting the irrelevant lines of text and using pythons string parsing functions to make a 2D list of the solar flare data. We use BeautifulSoup to get the text out of the 'a'  tags. Finally, we import this data into a pandas Dataframe df2, keeping only the important first 14 elements of each line. 

We replace all non-values of the various forms with NaNs and save this to a new dataframe df_nan.

## Step 4: Tidy NASA table
## To tidy this table, we replace all non-values of the various forms with NaNs and save this to a new dataframe df_nan. We then create a column of booleans called isHalo, making the value true if the corresponding row is of a halo flare.  We create another column width_lower_ bound of booleans, filled with True when row's width column contains a lower bound. Finally, we combine date and time columns as we did previously.

# PART 2 Analysis

## Question 1: Replication
#Can you replicate the top 50 solar flare table in SpaceWeatherLive.com #exactly using the data obtained from NASA? 
#That is, if you get the top 50 solar flares from the NASA table based on #their classification (e.g., X28 is the highest), do you get data for the ##same solar flare events?

No, if we just order by classification, we do not exactly replicate the data from SpaceWeatherLive.com using the data from NASA. The first reason is that the NASA data does not include the year 2017, while SpaceWeatherLive.com does. Also, some of the data from NASA has NaN values in the classification section.

[See the code below for the top 50 solar flare table from NASA]


## Question 2: Integration

#Write a function that finds the best matching row in the NASA data for each of the top 50 solar flares in the SpaceWeatherLive.com data. Here, you have to decide for yourself how you determine what is the best matchin entry in the NASA data for each of the top 50 solar flares.

Initial idea:
To find the best matching row in the NASA data for each of the top 50 solar flares on SpaceWeatherLive.com, we will look at matching start dates. Since X-class solar flares only occur about 10 times a year, if the start date of an X-class solar flare matches in the two data sets, then we can assume it is the same solar flare. 

Updated idea:
It turns out that the starting dates don't seem to match up perfectly. After doing some preliminary analysis, I've found that in the top 50 SpaceWeatherLive solar flares, there are no two solar flares of the same classification that also occur during the same year. Therefore, we will look for matches between the two data sets based on two conditions: they have the same classification AND, they occur during the same year. 


## Question 3: Analysis
Plot one plot that shows the top 50 solar flares in context with all data available in the NASA dataset.

I am going to plot the starting frequencies of the NASA dataset over time. 
Flares that appear in the SpaceWeatherLive set (and thus have a rank) will be distinguished by red dots. We will look to see if there is any clear trend as to how solar flare starting frequencies varies with the date & time of occurence. 



The covariance appears to be close to zero here. It looks like the starting frequency of the solar flares is independent of the time at which the solar flares occur. 